In [51]:
# We will be using Least square regression to give a linear model for the volatility of RMS. We derive a matrix X that is a design matrix 
# X represents 4 volatility factors over 5 time points (Feb 2026) derived from daily returns. They are lag 1 rolling volatilities 

import numpy as np

# 5x5 design matrix: intercept + 4 factors (BHP, RIO, FMG, ASX200)
X = np.array([
    [1.0, 0.0495, 0.0097, 0.0298, 0.0095],
    [1.0, 0.0205, 0.0012, 0.0229, 0.0153],
    [1.0, 0.0139, 0.0050, 0.0251, 0.0201],
    [1.0, 0.0151, 0.0150, 0.0265, 0.0309],
    [1.0, 0.0361, 0.0100, 0.0030, 0.0113]
])

# RMS rolling volatilities 
y_row = np.array([0.0226, 0.0168, 0.0076, 0.0329, 0.0550])
y = y_row.reshape(-1,1)

# Columns of X are rolling volatilities for BHP, RIO, FMG, ASX200 Market factor respectively. 
# Y is the rolling volatilities RMS. We orthogonally project Y onto factor space to minimise least square error
# We are hence deriving RMS volatility as a function of lagged volatilities of other stocks
# Factor subspace contains all volatility deviations that are explainable by those factors.

# Scale factor columns (columns 1-4) by 0.01 for interpretability
X_scaled = X.copy()
X_scaled[:,1:] = X[:,1:] / 0.01

# recall normal equation y = (X.T X)^-1 X.T y 

X_trans = X_scaled.T          # X^T
XtX = X_trans @ X_scaled      # X^T X
inv_XtX = np.linalg.inv(XtX)  # (X^T X)^(-1)
Xty = X_trans @ y             # X^T Y


Beta = inv_XtX @ Xty          # coefficents

B = Beta[0,0]
coef_BHP = Beta[1,0]
coef_RIO = Beta[2,0]
coef_FMG = Beta[3,0]
coef_ASX = Beta[4,0]

# Hence our linear regression reads
print("Linear regression is:", B, " +", coef_BHP, "BHP +", coef_RIO, "RIO +", coef_FMG, "FMG +", coef_ASX, "ASX")

# Our projection onto factor space reads
proj_y = X @ Beta

d1 = proj_y[0,0]
d2 = proj_y[1,0]
d3 = proj_y[2,0]
d4 = proj_y[3,0]
d5 = proj_y[4,0]
mag_proj = np.linalg.norm(proj_y)

print("Hence, for first period RMS volatility by factors and intercept is predicted as", -d1,
      "second period is", -d2,
      "third period is", -d3,                                              # the negative signs here are for cohessive interpretability
      "fourth period is", -d4,
      "fifth period is", -d5,
      "total volatility explained by factors is hence", mag_proj)

# Other statistical data obtained
idyo = y - proj_y

d1u = idyo[0,0]
d2u = idyo[1,0]
d3u = idyo[2,0]
d4u = idyo[3,0]
d5u = idyo[4,0]

mag_y = np.linalg.norm(y)
mag_idyo = np.linalg.norm(idyo)

print("Hence, volatility unexplained by factors for first period is", d1u,
      "second is", d2u,
      "third is", d3u,
      "fourth is", d4u,
      "fifth is", d5u,
      "Total volatility unexplained is", mag_idyo,
      "Total volatility of RMS is", mag_y)


Linear regression is: -0.061775757283532684  + 0.026491569013266325 BHP + -0.030385524932603047 RIO + -0.02221671225052546 FMG + 0.05149710774637484 ASX
Hence, for first period RMS volatility by factors and intercept is predicted as 0.06093199971069734 second period is 0.060989999710697344 third period is 0.061081999710697346 fourth period is 0.06082899971069735 fifth period is 0.060607999710697344 total volatility explained by factors is hence 0.13615063781286088
Hence, volatility unexplained by factors for first period is 0.08353199971069734 second is 0.07778999971069735 third is 0.06868199971069734 fourth is 0.09372899971069734 fifth is 0.11560799971069735 Total volatility unexplained is 0.19974090992782367 Total volatility of RMS is 0.07041427412108996


In [101]:
# Here, we construct a matrix whose columns are RMS, BHP, RIO over 3 periods of returns. We can take this to be a portfolio of sorts
# PCA via SVD on 3-stock portfolio to identify dominant variance directions and project daily returns onto these PCs for interpretation.
# This helps us reduce dimensions of our portfolio, we can analyse volatility contributions of stocks and reduce risk of portfolio.

import numpy as np

A = np.array([
    [-0.01745,  0.01205,  0.00325],  # 17 Feb: RMS, BHP, RIO
    [ 0.01549, -0.01045, -0.00233],  # 16 Feb
    [-0.03624,  0.00854,  0.00555]   # 13 Feb
])

# We are centering matrix of returns by the mean to capture variance rather than arbitary numerical percentages

A_centered = A - A.mean(axis=0)
cov_matrix = (A_centered.T @ A_centered) / (A.shape[0]-1)

eigvals, eigvecs = np.linalg.eig(cov_matrix)
eigvals[eigvals < 0] = 0                        # for this trivial project we are not concerned with eigenobjects over complex field
sv = np.sqrt(eigvals)
S = np.diag(sv)

v1 = eigvecs[:,0].reshape(-1,1)
v2 = eigvecs[:,1].reshape(-1,1)
v3 = eigvecs[:,2].reshape(-1,1)

e1 = v1/np.linalg.norm(v1)  
w2 = v2 - np.dot(v2.T,e1) * e1 
e2 = w2/np.linalg.norm(w2) 
w3 = v3 - np.dot(v3.T,e1)*e1 - np.dot(v3.T,e2)*e2
e3 = w3/np.linalg.norm(w3) 

# V.T gives us significant stock combinations in stock space that experience variance over the period

V = np.hstack([e1, e2, e3])
V_t = V.T

sv1 = S[0,0]
sv2 = S[1,1]
sv3 = S[2,2]

u1 = 1/sv1 * A @ e1
u2 = 1/sv2 * A @ e2
u3 = 1/sv3 * A @ e3

# This helps us with projections onto stock-return space to capture variance

U = np.hstack([u1, u2, u3])

# This follows from the well defined decompostion A = USV.T

print("Applying SVD to A_centered, we are left with the following decomposition: ")
print(A_centered, " == ")
print(U)
print(S)
print(V_t)

print("Our singular values are, ", sv1, "    ,", sv2, "and trivially, ", sv3)

# These are the main directions of stock space that experience well defined variance by singular values

pc1 = V_t[:,0].reshape(-1,1)
pc2 = V_t[:,1].reshape(-1,1)

pc12 = np.hstack([pc1, pc2])

print("Our principle components are, PC1: ", pc1, "   and PC2:", pc2)

# We are seeing how much we can reduce our dimensions
# Less abstractly, this answers, "How well do our combinations of stock components explain portfolios variance over period?"

pc1_proj = A_centered @ pc1
pc12_proj = A_centered @ pc12

print("Hence our projection of pc1 onto stock-return space is,", pc1_proj)
print("Hence our projection of pc2 onto stock-return space is,", pc12_proj)

# We see singular value of PC1 is 0.02 >> 0.005,0. Hence PC1 which is BHP dominated explains most variance in our portfolio over period (~80%)
# PC2 is RIO dominated, PC3 is RMS dombinated. Both have small singular values (hence RMS is trival in variance explainations 
# PC1 projection shows returns went minor but negative along PC1 (hence BHP domination) for period 1.
# PC1[2,1] = 0.030 implies returns moved strongly positive with PC1 and again hence BHP. PC1[3,1] = -0.02 shows returns moved strongly against PC1
# PC1 explains most variance. PC1+PC2 projection shows period 2,3 moved only slightly with PC2 (hence RIO), and against RIO for period 1. 
# Hence we have explained majority variability with BHP and RIO, this is dimension reduction of varability of returns. Hence we have applied PCA

Applying SVD to A_centered, we are left with the following decomposition: 
[[-0.00471667  0.00867     0.00109333]
 [ 0.02822333 -0.01383    -0.00448667]
 [-0.02350667  0.00516     0.00339333]]  == 
[[-0.73439591 -0.78859986         nan]
 [ 0.64584804  0.65878628         nan]
 [-1.2977471   1.13041953         nan]]
[[0.02860978 0.         0.        ]
 [0.         0.00548778 0.        ]
 [0.         0.         0.        ]]
[[ 0.91223336 -0.38440815 -0.1416357 ]
 [-0.38810693 -0.9216129   0.00163393]
 [ 0.13116139 -0.05347927  0.9899175 ]]
Our singular values are,  0.0286097812260974     , 0.005487775948552484 and trivially,  0.0
Our principle components are, PC1:  [[ 0.91223336]
 [-0.38810693]
 [ 0.13116139]]    and PC2: [[-0.38440815]
 [-0.9216129 ]
 [-0.05347927]]
Hence our projection of pc1 onto stock-return space is, [[-0.00752418]
 [ 0.03052531]
 [-0.02300112]]
Hence our projection of pc2 onto stock-return space is, [[-0.00752418 -0.00623573]
 [ 0.03052531  0.00213657]
 [-0.02300112